# 法規資料庫分析
- 資料來源：全國法規資料庫所收錄之中文法律資料檔下載 https://data.gov.tw/dataset/18289
- 下載 zip, 解開後有  FalV.xml
```
  <法規>
    <法規性質>憲法</法規性質>
    <法規名稱>中華民國憲法</法規名稱>
    <法規網址>https://law.moj.gov.tw/LawClass/LawAll.aspx?pcode=A0000001</法規網址>
    <法規類別>憲法</法規類別>
    <最新異動日期>19470101</最新異動日期>
    <生效日期>
    </生效日期>
    <生效內容><![CDATA[]]></生效內容>
    <廢止註記>
    </廢止註記>
    <是否英譯註記>Y</是否英譯註記>
    <英文法規名稱>Constitution of the Republic of China (Taiwan)</英文法規名稱>
    <附件 />
    <沿革內容><![CDATA[1.中華民國三十六年一月一日國民政府令公布
  中華民國三十六年十二月二十五日施行
  中華民國三十五年十二月二十五日國民大會通過]]></沿革內容>
    <前言><![CDATA[中華民國國民大會受全體國民之付託，依據孫中山先生創立中華民國之遺
教，為鞏固國權，保障民權，奠定社會安寧，增進人民福利，制定本憲法
，頒行全國，永矢咸遵。]]></前言>
    <法規內容>
      <編章節>   第 一 章 總綱</編章節>
      <條文>
        <條號>第 1 條</條號>
        <條文內容><![CDATA[中華民國基於三民主義，為民有民治民享之民主共和國。]]></條文內容>
      </條文>
    </法規內容>

  </法規>
  <法規>
```


<法規類別>行政＞勞動部＞組織目</法規類別>

## 基本列表與查詢
- 所有法規名稱的列表
- 指定法規列條文
- 客製化搜尋

本部分不建構管理物件，單純 parse 時做動作

In [ ]:
import xml.etree.ElementTree as ET

def parse_xml(xml_file,filter_key="",law_name=[]):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    cnt = 0
    for 法規 in root:
        法規性質 = 法規.find('法規性質').text
        法規名稱 = 法規.find('法規名稱').text
        法規網址 = 法規.find('法規網址').text
        法規類別 = 法規.find('法規類別').text
        # ... 其他欄位以此類推
        
        if not law_name:
          print(f"法規名稱: {法規名稱}")  
          cnt+=1
        else:
          if filter_key=="class":
            if 法規類別.find("行政＞勞動部")==-1:
              continue 
          elif 法規名稱 not in law_name:
            continue
          cnt+=1
          print(f"----- 法規名稱: {法規名稱} -----") 
          #print(f"法規類別: {法規類別}")
      
          if 1:
            法規內容 = []
            for 條文 in 法規.find('法規內容').findall('條文'):
              條號 = 條文.find('條號').text
              條文內容 = 條文.find('條文內容').text
              法規內容.append({ "條號": 條號, "條文內容": 條文內容})
              print(f"{條號}\n{條文內容}") 
            #print(f"法規內容: {法規內容}")
        #print(f"法規性質: {法規性質}")
        #print(f"法規名稱: {法規名稱}")
        #print(f"法規網址: {法規網址}")
        # ... 其他欄位以此類推
        #print(f"法規內容: {法規內容}")
    #print(f"共有 {cnt} 筆資料")

filepath ="yourpath"
parse_xml(filepath + "/FalV.xml","",['勞動基準法'])  # 將 "your_xml_file.xml" 替換成您的 XML 檔案路徑

## 一般化建構成 LawMgr
- 底層建構的方式，是一般化的，可延用
- 另建 Mgr 物件，針對這個應用作直覺的管理

In [123]:
import xml.etree.ElementTree as ET

class XMLElement:
    def __init__(self, tag, attrib, text, children):
        self.tag = tag
        self.attrib = attrib
        self.text = text
        self.children = children
        self.tags = {} # key: tag, value: text

    def __repr__(self):
        children_repr = ", ".join(repr(child) for child in self.children)
        return f"XMLElement(tag={self.tag}, attrib={self.attrib}, text={self.text}, children=[{children_repr}])"


                
def parse_xml_withobj(xml_file, filter_key="", law_name=[]): #一般化解析出 laws, 保留 XML 結構
    def element_to_object(element):
        children = [element_to_object(child) for child in element]
        return XMLElement(
            tag=element.tag,
            attrib=element.attrib,
            text=element.text.strip() if element.text else None,
            children=children
        )

    tree = ET.parse(xml_file)
    root = tree.getroot()
    laws = {}
    cnt = 0

    for 法規 in root:
        法規名稱 = 法規.find('法規名稱').text
        #print(f"法規名稱: {法規名稱}")  
        law = element_to_object(法規)
        #print(law)
        #print(law.attrib.get('法規名稱', ''))
        
        if not law_name:
            #print(law)
            cnt += 1
        else:
            if filter_key == "class":
                if "行政＞勞動部" not in law.attrib.get('法規類別', ''):
                    continue
            elif 法規名稱 not in law_name: #law.attrib.get('法規名稱', '') 
                continue
            #print(law)
            cnt += 1
        
        laws[法規名稱]=law
    
    return laws


def remove_chars(target_str, chars_to_remove):
    # 创建一个翻译表，将每个要移除的字符映射到 None
    translation_table = str.maketrans('', '', chars_to_remove)
    # 使用 translate 方法移除字符
    return target_str.translate(translation_table)

class LawMgr():
    def __init__(self,laws):
        self.laws = laws # key: 法規名稱， value: 法規物件
        self.law_items ={} # key: 法規名稱， value: 法條物件（key: 第 1 條，value: 條文）
        self.law_related = {} # key: 法規名稱， value: {key: 條文, value: 法規名稱}
        for law_name in laws.keys():
            self.law_related[law_name]={}
            law = laws[law_name]
            for child in law.children:
                law.tags[child.tag] = child.text #建構內部的 tags 欄位
                if child.tag=='法規內容': # 編章節 部分會被遺漏
                    #print(child)
                    law_items = {}
                    if 1:
                        法規內容 = child
                        nodes = []
                        self.find_all(法規內容,nodes,'條文')
                        
                        for 條文 in nodes:
                            條號 = 條文.children[0].text
                            條文內容 = 條文.children[1].text
                            law_items[條號] = 條文內容
                            #print(f"{條號}\n{條文內容}")
                    self.law_items[law_name] = law_items
        for law_name in laws.keys():
            self.get_law_related(law_name,True)

    def get_child_txt(self,law_name,tag):
        law = self.laws[law_name]
        for child in law.children:
            if child.tag == tag:
                return child.text
    def find_all(self,node,nodes,tag):
        
        if node.tag == tag:
            nodes.append(node)
            return
        for child in node.children:
            self.find_all(child,nodes,tag)
    
    # can be run after init
    def get_law_related(self,law_name,saved=False): # init
        items = self.law_items[law_name]
        law_names = self.laws.keys()
        for k,v in items.items():
            for test_law in law_names:
                if v.find(test_law)!=-1:
                    if saved:
                        if k not in self.law_related[law_name].keys():
                            self.law_related[law_name][k]=[]
                        self.law_related[law_name][k].append(test_law)
                    else:
                        print(f"{k} found {test_law}\n{v}")
                
    def show_law(self,law_name):
        print(self.laws[law_name].tags)
        print(self.law_items[law_name])
    def show_related(self,law_name): # 輸出 mermaid 語法
        # law_name --> 有關聯性的條文（全名）
        # 全名 --> 關聯法條
        # 回傳關聯法條 list
        ret = []
        if self.law_related[law_name]:
            rchars = ' （）'
            for k,v in self.law_related[law_name].items():
                law_d = remove_chars(law_name,rchars)
                disp_name = f"{law_name}{k}"
                disp_name = remove_chars(disp_name,rchars)

                print(f"{law_d}-->{disp_name}")
                
                for v1 in v:
                    print(f"{disp_name}-->{v1}")
                    ret.append(v1)
        return ret
          


## 載入 Mgr

In [ ]:
# 示例调用
filepath ="yourpath"
laws = parse_xml_withobj(filepath + '/FalV.xml', filter_key="", law_name=[]) #class, 勞動檢查法
lawmgr = LawMgr(laws) #轉成管理物件

## 使用範例

In [ ]:
#顯示某法，含條文
#lawmgr.show_law('勞動檢查法')

#顯示某個法的某欄位
#print(lawmgr.get_child_txt('勞動基準法','法規網址'))

#顯示某法的基本欄位
#print(lawmgr.laws['勞動檢查法'].tags)

#所有法的列表
#print(lawmgr.laws.keys())

#顯示某法的關聯法
#print(lawmgr.law_related['就業保險法'])

if 0: # 顯示關聯性，兩層展開(mermaid 語法), mermaid 有 500 行限制
    cnt = 0 
    law_list = ['工會法']
    print("graph TD")
    for law in law_list:
        ret1 = lawmgr.show_related(law)
        #print(f"ret={ret1}")

    for law in ret1:
        ret2 = lawmgr.show_related(law)
        #print(f"ret={ret2}")         
        
                
if 0:  # 顯示某法的某條文      
    law_name = "就業保險法"
    item_cnt = "第 2 條"
    print(f"{law_name} {item_cnt} :\n{lawmgr.law_items[law_name][item_cnt]}")

